# Fine-tuning example for MT5 on a custom prompt dataset using `pytorch`

In [ ]:
# !pip install transformers datasets sentencepiece pandas matplotlib rouge_score nltk

In [ ]:
!sudo apt-get install git-lfs


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Configuration

In [51]:
model_id="google/mt5-small"
dataset_id = "philschmid/prompted-germanquad"

## Preprocess dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset(dataset_id)

## inspect dataset to get an understanding of `max_input_length` and `max_target_length`

In [ ]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
df = ds["train"].to_pandas()
df.sample()

In [ ]:
df["targets"].map(lambda x: len(tokenizer.convert_ids_to_tokens(tokenizer(x).input_ids))).\
    hist()

In [ ]:
len(len_targets[len_targets > 128])

## tokenize dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
max_input_length = 512
max_target_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"], max_length=max_input_length, truncation=True
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["review_title"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = ds.map(preprocess_function, batched=True)


In [ ]:
# test size will be 15% of train dataset
test_size=.15
seed=33

processed_dataset = tokenized_datasets["train"].shuffle(seed=seed).train_test_split(test_size=test_size)
processed_dataset

# Train the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments
from huggingface_hub import HfFolder

per_device_train_batch_size = 8
per_device_eval_batch_size = 16
num_train_epochs = 8
learning_rate = 5.6e-5
# Show the training loss with every epoch
logging_steps = len(processed_dataset["train"]) // per_device_train_batch_size
model_name = model_id.split("/")[-1]
output_dir=f"{model_name}-prompted-germanquad"
hub_token = HfFolder.get_token() # or your token directly "hf_xxx"
hub_model_id = f'{model_id.split("/")[1]}-{dataset_id}-v1'
fp16=True

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    learning_rate=learning_rate,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_total_limit=3,
    fp16=fp16,
    predict_with_generate=True,
    logging_steps=logging_steps,
    load_best_model_at_end=True,
    metric_for_best_model="rougeLsum",
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_token=hub_token,   
)

In [ ]:
import nltk

nltk.download("punkt")

In [ ]:
import numpy as np
from nltk.tokenize import sent_tokenize


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
trainer.push_to_hub(commit_message="Training complete", tags="text2text")
